In [ ]:
from datarail.experimental_design import process_assay as pa

In [ ]:
df_well = pa.construct_well_level_df('input_file.csv')

In [ ]:
df_well = pa.add_dmso_control(df_well)

In [ ]:
fingerprint_drug = 'Staurosporine'
fingerprint_dose = 1

In [ ]:
dfr = pa.randomize_wells(df_well, fingerprints=['RL_A', 'RL_B', 'RL_C'],
                         fingerprint_drug=fingerprint_drug,
                         fingerprint_dose=fingerprint_dose)